# ENVIRONMENT

In [1]:
! python3.9 -m pip install tensorflow
! python3.9 -m pip install keras



In [ ]:
import tensorflow as tf

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras


from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# suppress scientific notation in Pandas
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_colwidth", 5000)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
pd.set_option("precision", 3)
pd.set_option("display.max_colwidth", None)

plt.rcParams["figure.figsize"] = [12, 10]
plt.rcParams["figure.dpi"] = 150

sns.set()
sns.set_context("notebook", rc={"lines.linewidth": 2.5})
sns.set_style("whitegrid")

import warnings

warnings.filterwarnings("ignore")

RANDOM_STATE = 1

In [10]:
original_df = pd.read_json("./src/amazon/reviews_Digital_Music_5.json", lines=True)
# df = original_df.copy()

In [ ]:
# Import data set as a data frame.
# Load packages for TensorFlow, Keras, and other libraries.
# EDA – to better understand the data structure.
# Remove unusual characters: uppercase letters, punctuation marks, emoticons, and non-English characters.
# Measure the vocabulary size using (bags of words, etc.).
# Explore N-Gram (unigram, bigram, and trigram) structures for usefulness in text analysis as applicable.
# Determine the word embedding and explore embedded words based on target inputs.
# Set the maximum sequence length based on the length of the longest sentence in the data set.
# Tokenize sentences into a list of tokens and remove stop words.
# Pad cleaned sentences to fit the maximum sequence length after each text sequence.
# Create an activation function filled with dense layers of the neural network.
# Split the data set into training and test sets or into training/test/validation sets.


In [16]:
# 1.	Analyzing the data and review length.

original_df.head(2)
original_df.info()
original_df.shape

# 2.	Creating a dictionary and applying it to remove extraneous characters.
# 3.	Deciding on a typical review length.
# 4.	Tokenization.
# 5.	Applying Tensorflow’s keras and layers methods
# 6.	Splitting the dataset into train and test
# 7.	Fitting the model
# 8.	Model evaluation and reporting.


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","It's hard to believe ""Memory of Trees"" came out 11 years ago;it has held up well over the passage of time.It's Enya's last great album before the New Age/pop of ""Amarantine"" and ""Day without rain."" Back in 1995,Enya still had her creative spark,her own voice.I agree with the reviewer who said that this is her saddest album;it is melancholy,bittersweet,from the opening title song.""Memory of Trees"" is elegaic&majestic.;""Pax Deorum"" sounds like it is from a Requiem Mass,it is a dark threnody.Unlike the reviewer who said that this has a ""disconcerting"" blend of spirituality&sensuality;,I don't find it disconcerting at all.""Anywhere is"" is a hopeful song,looking to possibilities.""Hope has a place"" is about love,but it is up to the listener to decide if it is romantic,platonic,etc.I've always had a soft spot for this song.""On my way home"" is a triumphant ending about return.This is truly a masterpiece of New Age music,a must for any Enya fan!",5,Enya's last great album,1158019200,"09 12, 2006"
1,AZPWAXJG9OJXV,5555991584,bethtexas,"[0, 0]","A clasically-styled and introverted album, Memory of Trees is a masterpiece of subtlety. Many of the songs have an endearing shyness to them - soft piano and a lovely, quiet voice. But within every introvert is an inferno, and Enya lets that fire explode on a couple of songs that absolutely burst with an expected raw power.If you've never heard Enya before, you might want to start with one of her more popularized works, like Watermark, just to play it safe. But if you're already a fan, then your collection is not complete without this beautiful work of musical art.",5,Enya at her most elegant,991526400,"06 3, 2001"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64706 entries, 0 to 64705
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewerID      64706 non-null  object
 1   asin            64706 non-null  object
 2   reviewerName    64529 non-null  object
 3   helpful         64706 non-null  object
 4   reviewText      64706 non-null  object
 5   overall         64706 non-null  int64 
 6   summary         64706 non-null  object
 7   unixReviewTime  64706 non-null  int64 
 8   reviewTime      64706 non-null  object
dtypes: int64(2), object(7)
memory usage: 4.4+ MB


(64706, 9)

In [ ]:
# For Task Two: Actually, it is pretty simple.  There are three text files, Amazon, IMDB, and Yelp.  You want to do a neural net to find the Sentiment index among the reviews.  These are the high-level ten basic steps:

# 1.	If you run a couple of FOR loops to translate characters (essentially “cleaning up”).   There will be 52 unique characters remaining.
# 2.	Plot a histogram to find the length of the reviews. Most of the 500 reviews will be under 150 characters in length.
# 3.	Create a dictionary for characters and tokens.
# 4.	Use arrays to hold each review and assign to it a list of one-hot-encoded characters.
# 5.	Code the output array to represent the review as positive (1) or negative (0).  The target value is binary.
# 6.	Add terminators to the end of the data to fill the fixed length strings.
# 7.	Using sklearn, numpy, and keras train a neural net.  Split data prior to training into test and validation.
# 8.	By reading through the data in the network, weights are adjusted until errors are minimized.  Each period or cycle is referred to as an epoch.
# 9.	Then, you run a sequential neural net using keras.Sequential()
# 10.	After using the training process iteratively, the model.evaluate(x, y) will confirm the reliability. 


# Introduction

You have seen the power of using data analytical techniques to help organizations make data-driven decisions and now want to extend these models into areas of machine learning and artificial intelligence. In this task, you will explore the use of neural networks and natural language processing (NLP).

In this task, you will choose a data file from the Web Links section. The available data sets are as follows:
- [Amazon Product Data set](http://jmcauley.ucsd.edu/data/amazon/)
- [UCSD Recommender Systems Data sets](https://cseweb.ucsd.edu/~jmcauley/datasets.html)
- UCI Sentiment Labeled Sentences Data set

For this task, you will build a neural network designed to learn word usage and context using NLP techniques. You will provide visualizations and a report, as well as build your network in an interactive development environment.

In the telecom munications industry, customers can choose from multiple service providers and actively switch from one provider to another. Customer churn is defined as the percentage of customers who stopped using a provider’s product or service during a certain time frame. In this highly competitive market, some telecommunications industries can experience average annual churn rates as high as 25 Given that it costs 10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many providers retaining highly profitable customers is the number one business goal. To reduce customer churn, telecom munications companies need to predict which customers are at high risk of churn.

As part of the “churn” project, executives would like to see consider a time series on revenue from the first years of operation. Once they understand any patterns in that data, they feel confident in understanding the impact of churn in current times. The given time series data records the daily revenue, in million dollars, during the first two years of operation.

**Data File being used:**
teleco_time_series.csv

**Data Dictionary:**
- The data set consists of 731 rows and two columns:
    - Day Day during first two years of operation
    - Revenue Revenue in million dollars
    
Review the data dictionary and considerations related to the raw data file you have chosen and prepare the data for time series modeling. You will then analyze that data set using time series modeling, create visualizations, generate forecasts, and deliver the results of your analysis.

# Part 0: Data Exploration

In [ ]:
df.head()


In [ ]:
df.shape


In [ ]:
df.info()


In [ ]:
pd.DataFrame(df.isnull().sum(), columns=["# NaNs"])


In [ ]:
f"{df.isnull().sum().sum():,} Total NaN Cells"


# Part I:  Research Question

## A.  Describe the purpose of this data analysis by doing the following:



### 1.  Summarize one research question that you will answer using neural network models and NLP techniques. Be sure the research question is relevant to a real-world organizational situation and sentiment analysis captured in your chosen dataset.



### 2.  Define the objectives or goals of the data analysis. Be sure the objectives or goals are reasonable within the scope of the research question and are represented in the available data.



### 3.  Identify a type of neural network capable of performing a text classification task that can be trained to produce useful predictions on text sequences on the selected data set.


# Part II:  Data Preparation


## B.  Summarize the data cleaning process by doing the following:



### 1.  Perform exploratory data analysis on the chosen dataset, and include an explanation of each of the following elements:


#### presence of unusual characters (e.g., emojis, non-English characters, etc.)



#### vocabulary size

#### proposed word embedding length

#### statistical justification for the chosen maximum sequence length

### 2.  Describe the goals of the tokenization process, including any code generated and packages that are used to normalize text during the tokenization process.



### 3.  Explain the padding process used to standardize the length of sequences, including the following in your explanation:











#### if the padding occurs before or after the text sequence

#### a screenshot of a single padded sequence

### 4.  Identify how many categories of sentiment will be used and an activation function for the final dense layer of the network.

### 5.  Explain the steps used to prepare the data for analysis, including the size of the training, validation, and test set split.

### 6.  Provide a copy of the prepared dataset.

# Part III:  Network Architecture



## C.  Describe the type of network used by doing the following:



### 1.  Provide the output of the model summary of the function from TensorFlow.

### 2.  Discuss the number of layers, the type of layers, and total number of parameters.


### 3.  Justify the choice of hyperparameters, including the following elements:



#### activation functions



#### number of nodes per layer



#### loss function



#### optimizer



#### stopping criteria



#### evaluation metric

# Part IV:  Model Evaluation



## D.  Evaluate the model training process and its relevant outcomes by doing the following:



### 1.  Discuss the impact of using stopping criteria instead of defining the number of epochs, including a screenshot showing the final training epoch.

### 2.  Provide visualizations of the model’s training process, including a line graph of the loss and chosen evaluation metric.


### 3.  Assess the fitness of the model and any measures taken to address overfitting.



### 4.  Discuss the predictive accuracy of the trained network.


# Part V:  Summary and Recommendations



## E.  Provide the code used to save the trained network within the neural network.


## F.  Discuss the functionality of your neural network, including the impact of the network architecture.


## G.  Recommend a course of action based on your results.



# Part VI: Reporting



## H.  Create your neural network using an industry-relevant interactive development environment (e.g., a Jupyter Notebook). Include a PDF or HTML document of your executed notebook presentation.


## I.  List the web sources used to acquire data or segments of third-party code to support the application.


## J.  Acknowledge sources, using in-text citations and references, for content that is quoted, paraphrased, or summarized.


## K.  Demonstrate professional communication in the content and presentation of your submission.

